In [1]:
from matplotlib import pyplot as plt
import numpy as np 

In [ ]:
# Read trakectories
# Load inputs
mesh = load_mesh(args.morphomovie)
mesh_bmap_tp = load(args.time_map)
trajectories = load(args.trajectories)
data = load(args.data)

# Get the min and max time in the data
min_time = min(data[name]["t"] for name in data)
max_time = max(data[name]["t"] for name in data)

# Make sure that the trajectory min max are ok as well
max_trjs = max(i for _, i in trajectories[0])
max_time = min(max_time, max_trjs)
min_trjs = min(i for _, i in trajectories[0])
min_time = max(min_time, min_trjs)


# Get the list of possible times from the morphomove
morphomovie_timepoints = np.array(list(mesh))
# Keep the ones on the constrain
morphomovie_timepoints = morphomovie_timepoints[
    np.logical_and(
        morphomovie_timepoints >= min_time, morphomovie_timepoints <= max_time
    )
]
# Get the number of timepoints for future reference
number_timepoints = len(morphomovie_timepoints)

# Get the empty reconstruction dictionary
reconstruction = {
    t: {str(n): [] for n in range(mesh[t]["elements"].shape[0])}
    for t in morphomovie_timepoints
}

# Do trajectory interpolation
for _, trajectory in enumerate(trajectories): # ix for the spline figure!
    # Get the trajectory as a dictionary to ease of access to values
    trajectory_dict = dict((t, n) for n, t in trajectory)
    # Get the raw data for each trajectory
    trajectory_raw_data = {}
    for frame_name in data:
        # Get the time for that timepoint
        timepoint = data[frame_name]["t"]

        # Skipt if we are outside time limits
        if not min_time <= timepoint <= max_time:
            continue

        # If the timepoint is not in the trajectory, get a mesh with same number of triangles
        timepoint = mesh_bmap_tp.get(timepoint, timepoint)
        # FIXME: Think this again!

        # # Check this! TODO:
        # if min(trajectory_dict) > timepoint:
        #     continue

        # Get the triangle of that trajectory at that timepoint
        # And at the value onto the raw data
        triangle = int(trajectory_dict[timepoint])
        value = data[frame_name]["z"][triangle]
        trajectory_raw_data.setdefault(timepoint, []).append(value)

    # Unpack the raw data into time ; value list
    # FIXME: Make this more consise
    x, y = [], []
    for t in trajectory_raw_data:
        for u in trajectory_raw_data[t]:
            x.append(t)
            y.append(u)

    # Get the points and get unique points with 8 decimal resolution
    points = np.column_stack((x, y))
    points = np.unique(points.round(8), axis=0)

    # Interpolation
    if args.method == "splines":
        # Perform the spline with vedo
        spline = vedo.Spline(points, smooth=0.4, res=number_timepoints + 1)
        i_points = spline.points()
        x_interpolated = i_points[:, 0]
        y_interpolated = i_points[:, 1]
    elif args.method == "polynomial":
        # Curve fitting
        coeff = np.polyfit(x, y, deg=2)
        f = np.poly1d(coeff)
        x_interpolated = np.linspace(min(x), max(x), len(morphomovie_timepoints))
        y_interpolated = list(f(u) for u in morphomovie_timepoints)

    elif args.method == "piecewise":
        # Interpolation
        x_interpolated = np.linspace(min(x), max(x), len(morphomovie_timepoints))
        y_interpolated = np.interp(x_interpolated, x, y)
    else:
        raise NotImplementedError
    
    # Make sure there are no negative values on
    y_interpolated[y_interpolated < 0] = 0
    
    # ####
    # # NOTE: Uncomment for this block of the spline figure
    # # AND ADD ix instead of _ on the loop!
    # if ix == 13234:
    #     dump("figures/slides/op_splines_x.pickle", x)
    #     dump("figures/slides/op_splines_y.pickle", y)
    #     dump("figures/slides/op_splines_x_interpolated.pickle", x_interpolated)
    #     dump("figures/slides/op_splines_y_interpolated.pickle", y_interpolated)
    #     exit()
    # ####

    # Add the trajectory onto the main reconstruction
    for timepoint in morphomovie_timepoints:
        # Get the closest value in the curve
        index = np.abs(x_interpolated - timepoint).argmin()
        value = y_interpolated[index]

        # Get the triangle
        timepoint = mesh_bmap_tp.get(timepoint, timepoint)
        triangle = str(int(trajectory_dict[timepoint]))

        # Add it to the empty dictionary
        reconstruction[timepoint][triangle].append(value)
